In [1]:
# Group Members: Ariel Liu, Yuqin Wang, Zihao Wang, Huanlun Li
# Starting Date: 11.21.18

# Our Project is mainly targeting on Twitter Crawling, allowing users to give in user input for specific tweet
# searches. We will make a word map according to the frequencies of words the user chooses to post in his/her recent
# tweets. We will input President Donald Trump's user ID as a demo in the end of our project.